# Recipe Recommender System

**Want a specific meal for within a certain budget for a specific number of people**

In [23]:
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

## Read in Files

### Grocery Similarity Model

In [24]:
data_matrix = pd.read_csv('../../data/05_model_output/data_matrix_sim.csv')

In [25]:
data_matrix.set_index('Unnamed: 0', inplace=True)

Because this was an item-item based similarity the output gives us items that are most like the other items. That means that when I put in parsley I get out other seasonings like the list shown below. In order to get things that go with ingredients we will need do do this by user-item similarity instead. Let's run this later. 

#### item - item 

In [26]:
print(data_matrix.loc['parsley'].nlargest(11))

parsley     1.000000
garlic      0.161545
onion       0.159124
cilantro    0.153014
lemon       0.151143
tomatoes    0.128387
thyme       0.121335
dill        0.119339
onions      0.118740
small       0.117934
kale        0.117458
Name: parsley, dtype: float64


#### user - item 

In [28]:
# Construct a new dataframe with the 10 closest neighbours (most similar)
# for each artist.
data_neighbours = pd.DataFrame(index=data_matrix.columns, columns=range(1,11))
for i in range(0, len(data_matrix.columns)):
    data_neighbours.ix[i,:10] = data_matrix.ix[0:,i].sort_values(ascending=False)[:10].index
# Get the artists the user has played.

In [29]:
known_user_likes = ['avocados', 'blueberries', 'bell pepper', 'onions', 'chicken', 'beef', 'chocolate']

In [30]:
# Construct the neighbourhood from the most similar items to the
# ones our user has already liked.
most_similar_to_likes = data_neighbours.loc[known_user_likes]
similar_list = most_similar_to_likes.values.tolist()
similar_list = list(set([item for sublist in similar_list for item in sublist]))

In [32]:
most_similar_to_likes

,1,2,3,4,5,6,7,8,9,10
avocados,avocados,cucumbers,bell peppers,bananas,strawberries,apples,raspberries,clementines,tomatoes,blueberries
blueberries,blueberries,strawberries,raspberries,spinach,avocado,banana,milk,blackberries,bananas,yogurt
bell pepper,bell pepper,onion,pepper,avocado,tomatoes,spinach,carrots,tomato,cucumber,strawberries
onions,onions,tomatoes,garlic,cilantro,peppers,carrots,tomato,lemon,banana,limes
chicken,chicken,cheese,milk,banana,bread,avocado,yogurt,strawberries,turkey,eggs
beef,beef,chicken,cheese,milk,banana,tomatoes,avocado,onion,tomato,carrots
chocolate,chocolate,mega,maple frozen dessert,hint,cheese,ice cream,almond,almond milk,avocado,milk


### Recipe Database

In [6]:
infile = open('../../data/03_processed/crf_ingred_dict','rb')
recipe_dict = pickle.load(infile)
infile.close()

In [7]:
infile = open('../../data/03_processed/crf_links_dict','rb')
recipe_links_dict = pickle.load(infile)
infile.close()

In [17]:
recipe_dict['farro salad with green beans, corn, and cherry tomatoes']

[{'qty': '1/2 ',
  'unit': 'pound',
  'name': 'green beans',
  'comment': 'trimmed and cut into 1 inch lengths'},
 {'qty': '2',
  'unit': 'cups',
  'name': 'corn kernels',
  'comment': 'fresh or frozen'},
 {'qty': '1/2 ',
  'unit': 'cup',
  'name': 'pearled or semi pearled farro',
  'comment': ''},
 {'qty': '3',
  'unit': 'tablespoons',
  'name': 'olive oil',
  'comment': 'or more to taste'},
 {'qty': '1',
  'unit': 'bunch',
  'name': 'scallions',
  'comment': 'trimmed and chopped'},
 {'qty': '1', 'unit': '', 'name': 'shallot', 'comment': 'very thinly sliced'},
 {'qty': '1/2 ',
  'unit': 'pint',
  'name': 'mixed yellow or red cherry tomatoes',
  'comment': 'halved or quartered if large'},
 {'qty': '2',
  'unit': 'tablespoons',
  'name': 'white wine vinegar',
  'comment': ''},
 {'qty': '2',
  'unit': 'tablespoons',
  'name': 'chopped fresh oregano marjoram or parsley',
  'comment': ''}]

In [9]:
recipe_links_dict['apple carrot cupcakes']

'https://www.simplyrecipes.com/recipes/apple_carrot_cupcake/'

### Grocery Prices 

In [10]:
grocery_prices_marianos = pd.read_csv('../../data/03_processed/grocery_prices_marianos_final.csv')

In [11]:
grocery_prices_marianos

,product,main_price,prod_aile,price_per_lb,measure_words_main_price,item_weight_count_vol,date_collected,store,location,product_new
0,Green Onions (Scallions),$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,scallions
1,Cilantro,$0.79,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,cilantro
2,Italian Parsley,$0.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,parsley
3,Roundy's Organic Fresh Rosemary,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,roundys
4,Roundy's Organic Fresh Thyme,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,thyme
5,Roundy's Mint,$2.50,Fresh Herbs,NaN,each,0.75 ounce,2019-08-28,Marianos,60615,mint
6,Basil,$1.79,Fresh Herbs,NaN,each,1 each,2019-08-28,Marianos,60615,basil
7,Dill,$1.99,Fresh Herbs,NaN,each,NaN,2019-08-28,Marianos,60615,dill
8,Roundy's Organics Fresh Dill,$2.50,Fresh Herbs,NaN,each,1 bunch,2019-08-28,Marianos,60615,dill
9,Gourmet Garden™ Ginger Stir-in Paste,$4.49,Fresh Herbs,NaN,each,4 ounce,2019-08-28,Marianos,60615,paste


***

In [12]:
def extract_ingredients(single_recipe_ing_list):
    ret = []
    for ingredient in single_recipe_ing_list:
        ret.append(ingredient['name'])
    return ret 

In [13]:
def find_recipe_by_ingredient(recipe_dict, search_item):
    
    listofkeys = []
    listofitems = recipe_dict.items()

    for item in listofitems:
        lst = extract_ingredients(item[1])
        subs = search_item
        if any(subs in s for s in lst):
            listofkeys.append(item[0])
    return listofkeys

In [14]:
print(grocery_similarities.loc['parsley'].nlargest(11))

parsley     1.000000
garlic      0.161545
onion       0.159124
cilantro    0.153014
lemon       0.151143
tomatoes    0.128387
thyme       0.121335
dill        0.119339
onions      0.118740
small       0.117934
kale        0.117458
Name: parsley, dtype: float64


In [15]:
find_recipe_by_ingredient(recipe_dict, 'chocolate')

['easy chocolate pudding in the microwave',
 'paleo hot cocoa',
 'cincinnati chili',
 'classic ice cream sandwiches',
 'red velvet cookies',
 'peanut butter pretzel magic bars',
 'peanut butter chocolate chip banana bread',
 'peanut butter blossoms',
 'oatmeal almond butter breakfast cookies',
 'thin and crispy chocolate chip cookies',
 'peanut butter swirl brownies',
 'brookies (chocolate chip brownie cookies)',
 'double chocolate cupcakes',
 'german chocolate snack cake with coconut-pecan frosting',
 's’mores cookies',
 'fudgy chocolate brownies',
 'chocolate banana bread',
 'chocolate bundt cake',
 'chocolate peanut butter bars',
 'walnut mocha torte',
 'white chocolate macadamia nut cookies',
 'cowboy cookies',
 'carmelitas',
 'peppermint bark',
 'chocolate ganache torte',
 'christmas crack',
 '7-layer magic bars']

In [16]:
recipe_dict['cincinnati chili']

[{'qty': '2',
  'unit': 'pounds',
  'name': 'ground beef',
  'comment': '80 20 is good'},
 {'qty': '1', 'unit': '6 ounce', 'name': 'can tomato paste', 'comment': ''},
 {'qty': '4', 'unit': 'cups', 'name': 'water', 'comment': ''},
 {'qty': '1', 'unit': '8 ounce', 'name': 'can tomato sauce', 'comment': ''},
 {'qty': '1',
  'unit': '',
  'name': 'large onion',
  'comment': 'minced about 3 cups'},
 {'qty': '6', 'unit': 'cloves', 'name': 'garlic', 'comment': 'minced'},
 {'qty': '3', 'unit': 'tablespoons', 'name': 'chili powder', 'comment': ''},
 {'qty': '1', 'unit': 'teaspoon', 'name': 'cumin', 'comment': ''},
 {'qty': '1', 'unit': 'teaspoon', 'name': 'cinnamon', 'comment': ''},
 {'qty': '3/4 ', 'unit': 'teaspoon', 'name': 'ground allspice', 'comment': ''},
 {'qty': '1/4 ', 'unit': 'teaspoon', 'name': 'ground cloves', 'comment': ''},
 {'qty': '1/2 ', 'unit': 'teaspoon', 'name': 'cayenne', 'comment': ''},
 {'qty': '2', 'unit': 'teaspoons', 'name': 'kosher salt', 'comment': ''},
 {'qty': '2',

## Pricing Recipes

## Recipe Recommendor System 